In [3]:
import pandas as pd
import numpy as np
import os
from glob import glob

def create_binary_labeling(csv_path):
    # Extract file name without extension
    file_name = os.path.basename(csv_path).replace('.csv', '')

    # Import raw data
    df = pd.read_csv(csv_path)
    
    # Organize a new df with relevant columns
    if 'Starting frame' not in df.columns or 'Ending frame' not in df.columns:
        print(f"Skipping {csv_path}: Required columns not found.")
        return None
    
    start_end = df[['Starting frame', 'Ending frame']]
    
    # Ensure indexes pair with number of rows
    start_end = start_end.reset_index(drop=True) 

    # Remove rows with NaN values
    start_end.dropna(axis=0, inplace=True)

    # Convert values to integers
    start_end = start_end.astype('int32')

    # Create list of frames where there was grooming
    grooming_frames = []
    for i in range(len(start_end)):
        grooming_frames.extend(range(start_end.loc[i, 'Starting frame'], start_end.loc[i, 'Ending frame'] + 1))

    # Create new df with binary labeling
    to_df = {'frames': list(range(1, 54000)), file_name: np.nan}
    grooming = pd.DataFrame.from_dict(to_df)

    # Create binary labeling for grooming column  
    grooming[file_name] = grooming['frames'].apply(lambda x: 1 if x in grooming_frames else 0)
    
    return grooming

# Folder containing CSV files
csv_folder = r'Z:\UriMons\Freezing_testing\automated_freezing_detection_test'
output_excel = r'Z:\UriMons\Freezing_testing\automated_freezing_detection_test\Bins_output\merged_behavior.xlsx'

# Get list of all CSV files in the folder
csv_files = glob(os.path.join(csv_folder, "*.csv"))

# Process each CSV file and store results
df_list = []
for csv_file in csv_files:
    df = create_binary_labeling(csv_file)
    if df is not None:
        df_list.append(df.set_index("frames"))  # Set frames as index for merging

# Merge all DataFrames by 'frames' (inner join to maintain alignment)
if df_list:
    merged_df = pd.concat(df_list, axis=1).reset_index()

    # Save merged DataFrame to a single Excel sheet
    merged_df.to_excel(output_excel, index=False)

    print(f"Processed {len(csv_files)} CSV files. Merged result saved to {output_excel}")
else:
    print("No valid CSV files found or processed.")


Skipping Z:\UriMons\Freezing_testing\automated_freezing_detection_test\M329_1080-1140sec_Cam1_freezing_bouts.csv: Required columns not found.
Skipping Z:\UriMons\Freezing_testing\automated_freezing_detection_test\M329_1080-1140sec_Cam2_freezing_bouts.csv: Required columns not found.
Skipping Z:\UriMons\Freezing_testing\automated_freezing_detection_test\M352_610-730sec_Cam1_freezing_bouts.csv: Required columns not found.
Skipping Z:\UriMons\Freezing_testing\automated_freezing_detection_test\M352_610-730sec_Cam2_freezing_bouts.csv: Required columns not found.
Skipping Z:\UriMons\Freezing_testing\automated_freezing_detection_test\M354_31000-34600f_Cam1_freezing_bouts.csv: Required columns not found.
No valid CSV files found or processed.
